In [ ]:
import numpy as np #importa a biblioteca usada para trabalhar com vetores de matrizes
%run normalization_utils.py

X, Y, feature_names = build_bow()

In [ ]:
%run normalization_utils.py

X_1000, Y, feature_names_1000, freq_list_1000 = build_reduced_bow(1000,
                                                                      'data/bow/reduced/best_words_1000.txt',
                                                                      'data/bow/reduced/best_frequency_1000.txt',
                                                                      'data/bow/reduced/reduced_bow_1000.npz',
                                                                      False)

X_5000, Y, feature_names_5000, freq_list_5000 = build_reduced_bow(5000,
                                                                      'data/bow/reduced/best_words_5000.txt',
                                                                      'data/bow/reduced/best_frequency_5000.txt',
                                                                      'data/bow/reduced/reduced_bow_5000.npz',
                                                                      False)

X_10000, Y, feature_names_10000, freq_list_10000 = build_reduced_bow(10000,
                                                                      'data/bow/reduced/best_words_10000.txt',
                                                                      'data/bow/reduced/best_frequency_10000.txt',
                                                                      'data/bow/reduced/reduced_bow_10000.npz',
                                                                      False)

X_b_1000, Y, feature_names_b_1000, freq_list_b_1000 = build_reduced_bow(1000,
                                                                              'data/bow/reduced_binary/best_words_1000.txt',
                                                                              'data/bow/reduced_binary/best_frequency_1000.txt',
                                                                              'data/bow/reduced_binary/reduced_bow_1000.npz',
                                                                              True)

X_b_5000, Y, feature_names_b_5000, freq_list_b_5000 = build_reduced_bow(5000,
                                                                              'data/bow/reduced_binary/best_words_5000.txt',
                                                                              'data/bow/reduced_binary/best_frequency_5000.txt',
                                                                              'data/bow/reduced_binary/reduced_bow_5000.npz',
                                                                              True)

X_b_10000, Y, feature_names_b_10000, freq_list_b_10000 = build_reduced_bow(10000,
                                                                              'data/bow/reduced_binary/best_words_10000.txt',
                                                                              'data/bow/reduced_binary/best_frequency_10000.txt',
                                                                              'data/bow/reduced_binary/reduced_bow_10000.npz',
                                                                              True)

In [ ]:
# Testando metodo para converter uma string numa amostra para predição
sentence = 'Tinha uma pedra no meio do caminho, no meio do caminho tinha uma pedra. Qual é o sentido da vida se a nossa existência é tão efemera.'

a = sentence_to_features(sentence, feature_names_1000)

print(a[a!=0])

In [ ]:
print('Calculando folds...')
folds = stratified_kfolds(Y, 5)
print('Folds calculados')

In [ ]:
Xtrain = X[folds[0][0]]
Ytrain = Y[folds[0][0]]

Xtest = X[folds[0][1]]
Ytest = Y[folds[0][1]]

In [ ]:
datasets = {
    'common': X,
    'reduced_1000': X_1000,
    'reduced_5000': X_5000,
    'reduced_10000': X_10000,
    'reduced_binary_1000': X_b_1000,
    'reduced_binary_5000': X_b_5000,
    'reduced_binary_10000': X_b_10000
}

keys = list(datasets.keys())

bestDatasets = {}

print(keys)

In [ ]:
%run Methods/Kneighbors.py
%run normalization_utils.py
%run saveAcc.py

bestAccuracy = 0
bestK = -1
bestDataset = ''

for key in keys:
    for k in range(1, 101, 2):
        print('%s bow - %d/%d' %(key, k, 101), end='\r', flush=True)
        acc = 0
        for i in range(5):
            Ypred = testSamples(datasets[key][folds[i][0]], Y[folds[i][0]], datasets[key][folds[i][1]], k)
            result = get_performance(Y[folds[i][1]], Ypred)
            acc = acc + result['acuracia']
        saveAcuracia(k, acc/5, 'Knn')
        if bestAccuracy < acc/5:
            bestAccuracy = acc/5
            bestK = k
            bestDataset = key

print('Best -> K: ', bestK, ' acc: ', bestAccuracy, 'dataset: ', bestDataset)

In [ ]:
%run Methods/logisticRegression.py
%run normalization_utils.py
%run saveAcc.py

bestAcuracia = 0
bestLambda = -1
bestDataset = ''
results = []

for key in keys:
    for i in range(0, 1001, 50):
        print('%s bow - %d/%d' %(key, i, 1001), end='\r', flush=True)
        acc = 0
        for j in range(5):
            theta, Ypred = regression(datasets[key][folds[j][0]], Y[folds[j][0]], datasets[key][folds[j][1]], i)
            result = get_performance(Y[folds[j][1]], Ypred)
            acc = acc + result['acuracia']
        saveAcuracia(i, acc/5)
        if bestAcuracia < acc/5:
            bestAcuracia = acc/5
            bestLambda = i
            bestDataset = key

print('Melhor Lambda: ', bestLambda, 'com acuracia:', bestAcuracia, 'dataset: ', bestDataset)

In [ ]:
%run Methods/naiveBayes.py
%run normalization_utils.py
%run saveAcc.py

bestAcuracia = 0
bestDataset = ''

results = []

for key in keys:
    print('Utilizando %s bow' %key, flush=True)
    acc = 0
    for i in range(5):
        Ypred = naiveBayes(datasets[key][folds[i][0]], Y[folds[i][0]],  datasets[key][folds[i][1]])
        result = get_performance(Y[folds[i][1]], Ypred)
        acc = acc + result['acuracia']
    results.append(acc/5)
#     saveAcuracia()
    if bestAcuracia < acc/5:
        bestAcuracia = acc/5
        bestDataset = key
        
bestDatasets['naive_bayes'] = key
print('Acuracia:', bestAcuracia, 'dataset: ', bestDataset)

In [ ]:
%run Methods/neuralNetworks.py
%run accuracy.py

vLambda = 1000

Ypred, Theta1, Theta2 = neuralNetwork(Xtrain, Ytrain, Xtest, vLambda)
display(accuracy(Ypred, Ytest))

In [ ]:
%run Methods/svmMethod.py
%run normalization_utils.py

kernel = 0
cost = 4
gamma = 1

model, Ypred = svmUse(Xtrain, Ytrain, Xtest, Ytest, kernel, cost, gamma)
result = get_performance(Ytest, Ypred)
print('cost:', cost, ' kernel:', kernel, ' gamma:', gamma, 'accuracy:', result['acuracia'])

In [ ]:
%run Methods/naiveBayes.py
%run normalization_utils.py

Xtrain = datasets[bestDatasets['naive_bayes']][folds[0][0]]
Ytrain = Y[folds[0][0]]

Xtest = datasets[bestDatasets['naive_bayes']][folds[0][1]]
Ytest = Y[folds[0][1]]

learning_curve(Xtrain,
               Ytrain,
               Xtest,
               Ytest,
               train,
               lambda X, result: [predict(X.getrow(i).toarray(),
                                          result[2],
                                          result[3],
                                          result[0],
                                          result[1])[0] for i in range(X.shape[0])])